In [1]:
from functions_collection_copy import *
import random as random
import statistics
import csv
from IPython.display import clear_output

def new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,start_node, check_integrity=False):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
    matrix_for_MST=matrix #copy the matrix, so that we ca keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    if start_node=='mid':
        starting_node=str(np.argmin(matrix.sum(axis=1)))
    elif start_node=='ex':
        starting_node=str(np.argmax(matrix.sum(axis=1)))
    elif start_node=='rd':
        starting_node=str(np.random.randint(0,matrix.shape[0]))
    elif start_node=="min_sn":
        starting_node=str(np.unravel_index(np.argmin(matrix), matrix.shape)[0])
    elif start_node=="max_sn":
        starting_node=str(np.unravel_index(np.argmax(matrix), matrix.shape)[0])
    elif start_node=='0':
        starting_node='0'
    else: 
        sys.exit()

    min_span_edges_res=find_min_span_edges_Prim(matrix_for_MST, starting_node)
    in_which_MSA_is_it={}
    names=set()
    for element in min_span_edges_res:
        print(element[2], element[3])
        names.add(element[2])
        names.add(element[3])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    MSA_list=[[[char] for char in seq] for seq in seqs]
    who_aligned_to_who=[]
    for row in min_span_edges_res:
        node1=row[2]
        node2=row[3]
        who_aligned_to_who.append([node1,node2])
    for element in min_span_edges_res:
            node1=element[2]
            node2=element[3]
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)])
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            united_MSA_new=alt_alt_merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2)
            in_which_MSA_is_it, MSA_list=perform_updates_OBO(in_which_MSA_is_it, node1, node2, united_MSA_new, MSA_list)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    if check_integrity==True:
       integrity_check_OBO_and_gradual(seqs,in_which_MSA_is_it,who_aligned_to_who,MSA_list, matrix,score_matrix,gap_cost)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(matrix,MSA_list, total_cost)



In [2]:
score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5
seqs=['aaatt',
 'cagtcgtt',
 'agctg',
 'ctgtagaac',
 'gcagg',
 'tttttt',
 'agcggcg',
 'cacccgtga',
 'tctgcag',
 'taactcata']

zero=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,'0', check_integrity=False)[2]

mid=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,'mid', check_integrity=False)[2]

ex=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,'ex', check_integrity=False)[2]

rd=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,'rd', check_integrity=False)[2]

min_sn=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,"min_sn", check_integrity=False)[2]

max_sn=new_assembly_Prim_starting_node_diff(seqs,score_matrix,gap_cost,"max_sn", check_integrity=False)[2]

print(zero,mid,ex,rd, min_sn,max_sn)



0 2
2 6
4 6
0 1
1 9
7 9
4 8
1 5
1 3



























0 2
2 6
4 6
1 2
1 9
7 9
4 8
1 5
1 3



























1 3
1 9
7 9
0 1
0 2
2 6
4 6
1 5
4 8



























7 9
1 9
0 1
0 2
2 6
4 6
1 5
4 8
1 3



























0 2
2 6
4 6
0 1
1 9
7 9
4 8
1 5
1 3



























0 2
2 6
4 6
0 1
1 9
7 9
4 8
1 5
1 3



























1568 1497 1547 1568 1568 1568


In [11]:
csv_file_path="testing_outputs/cost_different_starting_nodes_prim_10reps_10str_len150.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['str_len','type','cost'])

str_lens=[150]
for nr in str_lens:
    zero_cost=[]
    mid_cost=[]
    ex_cost=[]
    rd_cost=[]
    min_sn_cost=[]
    max_sn_cost=[]
    repeats=10
    nr_of_str=10
    len_str=nr
    random_sequences_collection=[]
    for i in range(0,repeats):
        random_pile=[]
        for j in range (0,nr_of_str):
            a_sequence=[]
            for k in range(0,len_str):
                a_sequence.append(random.choice(['a','c','t','g']))
            a_sequence=''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)  
    for random_pile in random_sequences_collection:
        zero=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'0', check_integrity=False)[2]
        zero_cost.append(zero)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'0',zero])
        clear_output(wait=True)

        mid=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'mid',check_integrity=False)[2]
        mid_cost.append(mid)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'mid',mid])
            clear_output(wait=True)

        ex=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'ex', check_integrity=False)[2]
        ex_cost.append(ex)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'ex',ex])
        clear_output(wait=True)

        rd=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'rd', check_integrity=False)[2]
        rd_cost.append(rd)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'rd',rd])
        clear_output(wait=True)
        
        min_sn=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'min_sn', check_integrity=False)[2]
        min_sn_cost.append(min_sn)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'min_sn',min_sn])
        clear_output(wait=True)

        max_sn=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'max_sn', check_integrity=False)[2]
        max_sn_cost.append(max_sn)
        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([len_str,'max_sn',max_sn])
        clear_output(wait=True)

    print("Mean cost str len ", str(nr)," zero:")
    print(statistics.mean(zero_cost))
    print("Mean cost str len ", str(nr),"mid:")
    print(statistics.mean(mid_cost))
    print("Mean cost str len ", str(nr)," ex:")
    print(statistics.mean(ex_cost))
    print("Mean cost str len ", str(nr)," rd:")
    print(statistics.mean(rd_cost))
    print("Mean cost str len ", str(nr)," min_sn:")
    print(statistics.mean(min_sn_cost))
    print("Mean cost str len ", str(nr)," max_sn:")
    print(statistics.mean(max_sn_cost))


    with open(csv_file_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['summary stats:'])
        csvwriter.writerow([nr,round(statistics.mean(zero_cost),2),round(statistics.mean(mid_cost),2),round(statistics.mean(ex_cost),2),round(statistics.mean(rd_cost),2),round(statistics.mean(min_sn_cost),2),round(statistics.mean(max_sn_cost),2)])
        

Mean cost str len  150  zero:
23826.8
Mean cost str len  150 mid:
23793.3
Mean cost str len  150  ex:
23838.9
Mean cost str len  150  rd:
23795.5
Mean cost str len  150  min_sn:
23826.8
Mean cost str len  150  max_sn:
23988


In [10]:
csv_file_path="testing_outputs/cost_different_starting_nodes_prim_15reps_strlen50_diff_nr_of_str.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['str_nr,zero,mid,ex,rd,min_sn,max_sn'])

str_lens=[50]
nr_of_str_list=[3,5,10,20]
for nr_of_str in nr_of_str_list:
    for nr in str_lens:
        zero_cost=[]
        mid_cost=[]
        ex_cost=[]
        rd_cost=[]
        min_sn_cost=[]
        max_sn_cost=[]
        repeats=10
        len_str=nr
        random_sequences_collection=[]
        for i in range(0,repeats):
            random_pile=[]
            for j in range (0,nr_of_str):
                a_sequence=[]
                for k in range(0,len_str):
                    a_sequence.append(random.choice(['a','c','t','g']))
                a_sequence=''.join(a_sequence)
                random_pile.append(a_sequence)
            random_sequences_collection.append(random_pile)  
        for random_pile in random_sequences_collection:
            zero=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'0', check_integrity=False)[2]
            zero_cost.append(zero)
            clear_output(wait=True)

            mid=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'mid',check_integrity=False)[2]
            mid_cost.append(mid)
            clear_output(wait=True)

            ex=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'ex', check_integrity=False)[2]
            ex_cost.append(ex)
            clear_output(wait=True)

            rd=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'rd', check_integrity=False)[2]
            rd_cost.append(rd)
            clear_output(wait=True)
            
            min_sn=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'min_sn', check_integrity=False)[2]
            min_sn_cost.append(min_sn)
            clear_output(wait=True)

            max_sn=new_assembly_Prim_starting_node_diff(random_pile,score_matrix,gap_cost,'max_sn', check_integrity=False)[2]
            max_sn_cost.append(max_sn)
            clear_output(wait=True)

        print("Mean cost str len ", str(nr_of_str)," zero:")
        print(statistics.mean(zero_cost))
        print("Mean cost str len ", str(nr_of_str),"mid:")
        print(statistics.mean(mid_cost))
        print("Mean cost str len ", str(nr_of_str)," ex:")
        print(statistics.mean(ex_cost))
        print("Mean cost str len ", str(nr_of_str)," rd:")
        print(statistics.mean(rd_cost))
        print("Mean cost str len ", str(nr_of_str)," min_sn:")
        print(statistics.mean(min_sn_cost))
        print("Mean cost str len ", str(nr_of_str)," max_sn:")
        print(statistics.mean(max_sn_cost))


        with open(csv_file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([nr_of_str,round(statistics.mean(zero_cost),2),round(statistics.mean(mid_cost),2),round(statistics.mean(ex_cost),2),round(statistics.mean(rd_cost),2),round(statistics.mean(min_sn_cost),2),round(statistics.mean(max_sn_cost),2)])
            

KeyboardInterrupt: 